In [3]:
import os
import cv2
import joblib
import numpy as np
import pandas as pd
import mediapipe as mp
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from scipy.signal import savgol_filter


mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

def smooth_data(data, window_size=5, polyorder=2):
    if len(data) < window_size:
        return data  # No suficiente data para suavizar
    return savgol_filter(data, window_size, polyorder, axis=0)

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

def calculate_distance(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

def extract_pose_features(video_path, model, scaler_X, encoder):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        alto_original, ancho_original = frame.shape[:2]
        nuevo_alto = 1100  # Nueva altura deseada
        nuevo_ancho = int((nuevo_alto / alto_original) * ancho_original)
        frame = cv2.resize(frame, (nuevo_ancho, nuevo_alto))

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            keypoints = [(lm.x, lm.y, lm.z) for lm in landmarks]

            # Extract relevant angles and distances
            left_shoulder = keypoints[11]
            right_shoulder = keypoints[12]
            left_elbow = keypoints[13]
            right_elbow = keypoints[14]
            left_wrist = keypoints[15]
            right_wrist = keypoints[16]
            left_hip = keypoints[23]
            right_hip = keypoints[24]
            left_knee = keypoints[25]
            right_knee = keypoints[26]
            left_ankle = keypoints[27]
            right_ankle = keypoints[28]

            # Example angles
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            
            # Example distances
            wrist_distance = calculate_distance(left_wrist, right_wrist)
            ankle_distance = calculate_distance(left_ankle, right_ankle)
            shoulder_distance = calculate_distance(left_shoulder, right_shoulder)
            hip_distance = calculate_distance(left_hip, right_hip)

            features = np.array(keypoints).flatten().tolist()

            X = scaler_X.transform(np.array(features).reshape(1, -1))
            X = smooth_data(X)

            predictions = model.predict(X)

            # Separar las predicciones
            predicciones_continuas = predictions[0][0]
            prediccion_clase = encoder.inverse_transform(np.argmax(predictions[1], axis=1))

            features_video = [left_elbow_angle, right_elbow_angle, left_shoulder_angle, right_shoulder_angle, left_knee_angle, right_knee_angle, wrist_distance, ankle_distance, shoulder_distance, hip_distance]


            # Comparar características del video con las predichas
            porcentaje_diferencia = 0.2
            caracteristicas = ["Angulo del codo izquierdo", "Angulo del codo derecho", "Angulo del hombro izquierdo", "Angulo del hombro derecho", "Angulo de la rodilla izquierda", "Angulo de la rodilla derecha", "Distancia entre munecas", "Distancia entre tobillos", "Distancia entre hombros", "Distancia entre caderas"]


            array1 = np.array(predicciones_continuas)
            array2 = np.array(smooth_data(features_video))

            relacion = np.abs(1 - (array1 / array2))

            errores = [caracteristicas[i] for i in range(len(relacion)) if porcentaje_diferencia< relacion[i] < (1-porcentaje_diferencia)]


            # Dibujar la pose
            annotated_image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
            )

            clase_predicha = prediccion_clase[0]
            porcentaje_prediccion = np.max(predictions[1]) * 100

            # Mostrar la predicción de clase
            texto_clasificacion = f'Clasificacion: {clase_predicha} - Probabilidad: {porcentaje_prediccion:.2f}%'
            cv2.putText(annotated_image, texto_clasificacion, (50, 220), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            if len(errores) > 0:
                for i,error in enumerate(errores):
                    desplazamiento_vertical = 270 + (i * 30)  # Incrementa la posición vertical para cada error adicional
                    cv2.putText(annotated_image, f'Error en : {error}', (50, desplazamiento_vertical), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                print(relacion)

            cv2.imshow('Frame', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

            # Escuchar tecla 'q' para salir
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()



# Cargar el modelo y las clases
ruta_actual = os.getcwd()
carpeta_mediapipe = os.path.dirname(ruta_actual)
ruta_raiz = os.path.dirname(carpeta_mediapipe)

ruta_dataset = os.path.join(ruta_raiz, 'Data Set') 
ruta_video = os.path.join(ruta_dataset, 'test_video_4.mp4')
ruta_npy = os.path.join(ruta_actual, 'datos', 'classes.npy')
ruta_scaler = os.path.join(ruta_actual,'datos','scaler_X.pkl')
ruta_modelo = os.path.join(ruta_actual, 'model', 'exercise_model.h5')



# Cargar el modelo y scaler
model = load_model(ruta_modelo)
scaler_X = joblib.load(ruta_scaler)

classes = np.load(ruta_npy, allow_pickle=True)
encoder = LabelEncoder()
encoder.classes_ = classes

# Realizar la predicción
extract_pose_features(ruta_video,model,scaler_X,encoder)



c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
[0.09075414 0.79901022 0.0640529  0.22617722 0.2447809  0.34686022
 0.99065959 1.01989815 1.0122104  0.97943234]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[0.12914143 0.72414363 0.05995242 0.35423732 0.19883338 0.27194216
 0.99153875 1.0200143  1.01149426 0.97941855]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[0.13649112 0.70287094 0.07652542 0.37464639 0.18491896 0.26081567
 0.99149148 1.01989411 1.01142481 0.9793476 ]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[0.14152156 0.66622533 0.11281615 0.38530376 0.17616322 0.26339711
 0.99136023 1.01989996 1.01142707 0.97937294]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

[0.12022625 0.38152045 0.13309158 0.16081522 0.08142132 0.35048721
 0.98374071 1.02253954 1.01306663 0.98652336]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[0.11393627 0.4399693  0.12043273 0.23166472 0.08068769 0.32352119
 0.9846176  1.02295848 1.01277085 0.99436018]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step

c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[0.1190015  0.50954289 0.12024288 0.25096282 0.09716084 0.33956796
 0.98610788 1.02042583 1.01209448 0.99351711]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[0.13280362 0.52722233 0.09826417 0.28482867 0.11167601 0.34811567
 0.98682486 1.01963554 1.01194565 0.99002699]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[0.15132154 0.58062443 0.10459359 0.30133737 0.12173073 0.33768807
 0.98735688 1.01868679 1.0116191  0.98770874]
